In [ ]:
# Created by Samuele Buosi on 18/4/19

In [1]:
#import library
import numpy as np
import cv2
import pyyolo

/home/sam/anaconda3/envs/darknetpy_env/lib/python3.7/site-packages/pyyolo/libdarknet.so


In [2]:
# network loading
meta_filepath = b"/home/sam/Desktop/peopleDetector/darknet/cfg/coco.data"
cfg_filepath = b"/home/sam/Desktop/peopleDetector/darknet/cfg/yolov3Pred.cfg"
weights_filepath = b"/home/sam/Desktop/peopleDetector/darknet/yolov3.weights"

meta = pyyolo.load_meta(meta_filepath)
net = pyyolo.load_net(cfg_filepath, weights_filepath, False)

In [5]:
# blur BBOX
#Open video
cap = cv2.VideoCapture('/home/sam/Desktop/Video_Fergus/matrimonio/DJI_0315.MOV')
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
        
    # BONUS: resize frame, non necessario     
    frame = cv2.resize(frame, (768, 576))

    yolo_img = pyyolo.array_to_image(frame)
    res = pyyolo.detect(net, meta, yolo_img)
    
    
    for r in res:
        # skip se non e' la classe desiderata: person
        if r.name != b'person':
            continue
            
        # test
        #print(r.bbox)
        
        cv2.rectangle(frame, r.bbox.get_point(pyyolo.BBox.Location.TOP_LEFT, is_int=True),
                      r.bbox.get_point(pyyolo.BBox.Location.BOTTOM_RIGHT, is_int=True), (0, 255, 0), 2)

        #Blur BBOX
        # seleziono la parte del frame che voglio oscurare 
        pto1 = r.bbox.get_point(pyyolo.BBox.Location.TOP_LEFT, is_int=True)
        x1=pto1[0]
        y1=pto1[1]
        pto2 = r.bbox.get_point(pyyolo.BBox.Location.BOTTOM_RIGHT, is_int=True)
        x2=pto2[0]
        y2=pto2[1]
        # lasse y e' invertito!!!
        h = y2-y1
        w = x2-x1
        x3= x1 + (w//4)
        y3= y1 + (h//19)
        x4= x3 + (w//2)
        y4= y3 + (h//6)
        x5 = x2
        y5= y1 + h//4
        # img[y:y+h, x:x+w]
        #imgmask = frame[y1:y2, x1:x2] # all rect
        #imgmask = frame[y3:y4, x3:x4] # a square
        imgmask = frame[y1:y5, x1:x5]
        
        # applico un filtro per il blur
        imgmask = cv2.GaussianBlur(imgmask,(15,15),0)

        # inserisco la parte oscurata nel video, ATTENZIONE che le dimensioni siano giuste
        #frame[y1:y2, x1:x2] = imgmask # all rect
        #frame[y3:y4, x3:x4] = imgmask # a square
        frame[y1:y5, x1:x5] = imgmask
            
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Blur face + BBox
# Double detector at the same time (too slow for real time application)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
fullbody_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml.xml')

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
        
    # BONUS: resize frame, non necessario     
    frame = cv2.resize(frame, (1024, 1024))

    yolo_img = pyyolo.array_to_image(frame)
    res = pyyolo.detect(net, meta, yolo_img)
   
    for r in res:
        cv2.rectangle(frame, r.bbox.get_point(pyyolo.BBox.Location.TOP_LEFT, is_int=True),
                      r.bbox.get_point(pyyolo.BBox.Location.BOTTOM_RIGHT, is_int=True), (0, 255, 0), 2)
        
        # pti di riferimento ROI
        pto1 = r.bbox.get_point(pyyolo.BBox.Location.TOP_LEFT, is_int=True)
        x1=pto1[0]
        y1=pto1[1]
        pto2 = r.bbox.get_point(pyyolo.BBox.Location.BOTTOM_RIGHT, is_int=True)
        x2=pto2[0]
        y2=pto2[1]
        
        # seleziono porzione di immagine da analizzare per il blur faccia
        roi = frame[y1:y2, x1:x2]
     
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        print(faces)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            #roi_color = img[y:y+h, x:x+w]
            
            
            
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()